# Accessing texts from NB digital

Texts are accessed and analysed using tools built in the programming language Python. The problem is how to analyse texts when they are protected from distribution. One solution is to not challenge the copyright. Instead features of texts can be downloaded and used for analysis.

In [1]:
import dhlab.nbtext as nb
import dhlab.module_update as mu
import dhlab.graph_networkx_louvain as gnl
%matplotlib inline
mu.css()

In [2]:
mu.update('collocations', silent=True)
import collocations as col
import pandas as pd

# Find texts and search in them

Use properties of texts to constrain number of possibilities. Retrieve a selection of books that satisfies those contrainst. This is your corpus. The command is `book_corpus()`, which takes a number of parameters. Corpora can be joined, subtracted and combined in a number of ways. 

An output from a search `nb.book_corpus(author='Røssaak%Eivin%')`

In [3]:
corpus = nb.book_corpus(author='Røssaak%Eivin%')
corpus

,urn,author,title,year
0,2008061800060,"Røssaak, Eivind","Skrujern, kanoner og trojanske hester",1996
1,2008081400081,"Røssaak, Eivind",Det postmoderne og de intellektuelle,1998
2,2014072405078,"Røssaak, Eivind",Selviakttakelse,2005
3,2011032206046,"Røssaak, Eivind",(Sic),2001


This corpus can be searched, for example with the command 

`nb.urn_concordance(word=['postmodernisme', 'postmodernismen'], urns=corpus, before=10, after=10)`

In [4]:
nb.urn_concordance(word=['postmodernisme', 'postmodernismen'], urns=corpus, before=10, after=10)

"Skrujern, kanoner og trojanske hester, Røssaak, Eivind, 1996",en flora ny-ismer og post-ismer : modernisme er blitt til,postmodernisme,", strukturalisme til poststrukturalisme , pragmatisme til nypragmatisme , og"
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",et essay av Derrida som skyteskive og bevis for at,postmodernisme,og dekonstruksjon er galskap . I en nærlesning av det
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998","distinksjoner : « postmodernitet » brukes om en samfunnstilstand, «",postmodernisme,"» om spesielle retninger innen kunst , litteratur , arkitektur"
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",« post war » litteratur i negativ forstand som «,postmodernisme,» . 6 De sammenligner det nye med sin egen
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998","ny teori , og i USA kalles begge deler «",postmodernisme,» . Denne tilstandens første affirmative teoretikere var de kvinnelige
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",den første ( 1971 ) som bruker selve ordet «,postmodernisme,"» på det nye i positiv forstand , først og"
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",To store 68 - er-grupperinger har reagert svært negativt på,postmodernismen,. De er enten maoister eller maoistenes motstandere på venstresiden
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",konservativ reaksjon mot en ny fiende ; en karrikatur av,postmodernismen,. Jeg har forsøkt å finne noen av de rommene
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",De postmoderne ville « korrigere modernismens eksesser » . Etter,postmodernismen,"kom , ifølge Enrico , en « ultramodernismo » som"
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",definert identitet og en eksistensiell trygghet . For Bell oppstår,postmodernismen,der den « dekadente » modernismen ( det kan her
"Det postmoderne og de intellektuelle, Røssaak, Eivind, 1998",som forsvarer moderniteten mot dem han kaller nykonservative . Under,postmodernismen,"ønsker disse , mener han , å frigjøre seg fra"


What is connected with the term *postmodern* in the digital collection? Let us start with a larger corpus, one that contains one version of the word or the other

In [5]:
capitalize = lambda words: [x.capitalize() for x in words]

In [6]:
postmodern_words = "postmodernisme postmodernismen postmoderne".split()
postmodern_words += capitalize(postmodern_words)
postmodern_words

['postmodernisme',
 'postmodernismen',
 'postmoderne',
 'Postmodernisme',
 'Postmodernismen',
 'Postmoderne']

In [7]:
postmodern = nb.book_corpus(words= postmodern_words, limit = 20000)


In [8]:
mu.printmd('Number of books satisfying the postmodern condition: ' + "**" + str(len(postmodern))+ "**.")
postmodern.head()

Number of books satisfying the postmodern condition: **8906**.

,urn,author,title,year
0,2009100600108,,Samfunnsfag og samfunnsliv,1997
1,2012102907093,"Økland, Ståle",TrendMania,2011
2,2016093008061,"Eriksen, Thomas Hylland",Øyeblikkets tyranni,2002
3,2011030108040,"Rosenberg, Tone Birgit",Eldres opplevelser av det å ha trykksår,2000
4,2011031603060,"Fredheim, Arnt Normann",Steinar Christensens Refsnes-frise,2009


In [9]:
speedkonk = lambda søkeord, input_korpus, before=10, after=10: nb.urn_concordance(word=søkeord, urns=nb.refine_book_urn(urns=nb.pure_urn(input_korpus), words=søkeord, limit=10), after=after, before=before,)

In [10]:
speedkonk('litteratur', postmodern)

"Hvordan påvirker veiledningssamtalen studentens kunnskapsutvikling?, Eikeland, Anne, 2003","er , har vår nysgjerrighet og undring bidratt til at",litteratur,knyttet til læring gjennom dialog får mer fokus i arbeidet
"Hvordan påvirker veiledningssamtalen studentens kunnskapsutvikling?, Eikeland, Anne, 2003",I mye,litteratur,blir påstandskunnskapen knyttet opp mot et naturvitenskaplig / teknologisk kunnskapsyn
"Hvordan påvirker veiledningssamtalen studentens kunnskapsutvikling?, Eikeland, Anne, 2003",belyse refleksjonene våre om de flinke pikene med utgangspunkt i,litteratur,av Nygaard og Sand ( 1998 ) .
"Hvordan påvirker veiledningssamtalen studentens kunnskapsutvikling?, Eikeland, Anne, 2003",opplever det har alle våre valg hatt konsekvenser . Hvilken,litteratur,vi har lest har hatt konsekvenser for hvordan vi har
"Hvordan påvirker veiledningssamtalen studentens kunnskapsutvikling?, Eikeland, Anne, 2003",og økt bevissthet . Vi valgte å fordype oss i,litteratur,"om ulike kunnskapssyn , kommunikasjon og læring . Dersom vi"
"norske vietnambevegelsen 1967-1973, Godbolt, James, 2008",. 787 1 tillegg hadde Solkom til salgs et utvalg,litteratur,"for de mest avanserte aktivistene , bl.a. ei rekke o"
"Etikk, Eide, Solveig Botnen, 2005","engelsk « whistle blowing » , og det finnes også",litteratur,som omtaler dette som « fløyteblåsing » . I Ytringsfrihetskommisjonens
"moderne kvinnene i Odda, Danielsen, Hilde, 2000",hjemmene ødelagt av de store barneflokker . Kontoret vil forhandle,litteratur,"vedrørende disse spørsmål , samt de forskjellige preventiver ( Hardanger"
"Guide til sosiologi hovedfag, , 1996",og andre spesialseminar er det noen ganger tatt med anbefalt,litteratur,og tentative forelesningsplaner .
"Guide til sosiologi hovedfag, , 1996",gi rom for diskusjon av studentenes oppgåver og av relevant,litteratur,". """
"Guide til sosiologi hovedfag, , 1996",1. Om seminarplanen : Forslag til forelesningstemaer og,litteratur,", planlegging av studentinnlegg ."


In [35]:
col.dist_coll_urn??

In [37]:
pm = dict()
urnlist = nb.pure_urn(postmodern)
step = 500
for i in range(0, len(postmodern), step):
    pm[i] = col.urn_coll('postmodernismen',urns=urnlist[i:i+step], before=0, after=10)

In [38]:
pm_tot = pd.concat([pm[i] for i in pm])

In [40]:
pm_tot['weighted_distance'] = pm_tot.freq*pm_tot.dist

In [41]:
pm_tot.loc['han']

,freq,dist,weighted_distance
han,5.0,6.500000,32.500000
han,2.0,8.500000,17.000000
han,5.0,8.200000,41.000000
han,4.0,7.000000,28.000000
han,9.0,7.640000,68.760000
han,5.0,7.125000,35.625000
han,2.0,8.500000,17.000000
han,8.0,5.857143,46.857143
han,1.0,2.000000,2.000000
han,5.0,5.400000,27.000000


In [42]:
pm_total = pm_tot.groupby(pm_tot.index).sum()

In [45]:
pm_total.dist = pm_total.weighted_distance/pm_total.freq

In [53]:
pm_total.dist = round(pm_total.dist, 2)

In [57]:
pm_total['score'] = round(col.dist(pm_total['dist'], col.calculate_midpoint(0, 10), pm_total['freq']), 2)

In [54]:
col.dist_coll_urn??

In [59]:
pm_total.sort_values(by='score')

,freq,dist,weighted_distance,score
forfekter,5.0,1.00,5.00,1.90
tolkes,4.0,1.00,4.00,2.12
slo,8.0,1.75,14.00,2.22
dessuten,8.0,1.88,15.00,2.33
velkommen,5.0,1.60,8.00,2.38
generelt,18.0,2.22,40.00,2.40
snakker,3.0,1.00,3.00,2.50
fant,4.0,1.50,6.00,2.50
roper,3.0,1.00,3.00,2.50
oppfatter,3.0,1.00,3.00,2.50


In [62]:
def velg_form(word):
    former = list({x.split()[0] for x in nb.word_form(word)})
    if 'det' in former:
        res = 'det'
    elif 'pron' in former:
        res = 'pron'
    elif 'verb' in former:
        res = 'verb'
    elif 'subst' in former:
        res = 'subst'
    elif 'adj' in former:
        res = 'adj'
    else:
        try:
            res = former[0]
        except:
            res = 'ukjent'
    return res

def klassifikasjon(df, by='score', num = 60, up=True):
    gruppering = list(map(lambda w: (w, velg_form(w)), df.sort_values(by=by, ascending=up).head(num).index))
    grupper = dict(gruppering)
    grupperinger = {cat:[w for w in grupper if grupper[w] == cat] for cat in grupper.values()}
    return nb.frame(grupperinger).transpose().fillna('')

In [69]:
klassifikasjon(pm_total, num=120, up=True).head(30)

,verb,adv,subst,adj,ukjent,prep,fork
0,forfekter,dessuten,velkommen,generelt,viderefører,ned,bl.a.
1,tolkes,neppe,musikken,primært,poststrukturalismen,utover,
2,slo,derimot,samfunnsvitenskapene,trolig,Perspektivismen,hinsides,
3,snakker,punktvis,konstruktivismen,foreløpig,Sosiologisk,,
4,fant,,slutten,førebels,konseptualismen,,
5,roper,,næring,markant,273,,
6,oppfatter,,skisma,,dekonstruktivismen,,
7,feide,,billedkunsten,,Eg,,
8,fremviser,,barnet,,dekonstruksjonen,,
9,angriper,,konstruktivisme,,James,,


In [63]:
tot = nb.frame(nb.totals(50000), 'tot')

In [64]:
pm_total['pmi'] = (pm_total['freq']**1.5/tot['tot'])

In [68]:
klassifikasjon(pm_total, by='pmi', num=120, up=False).head(30)

,ukjent,subst,verb,adj
0,postmoderne,arkitekturen,oppløse,psykoanalytisk
1,Samtiden,arkitektur,vektlegger,universell
2,Foucault,diskurser,avviser,binære
3,Torben,sannheter,representerer,arkitektoniske
4,Habermas,epoker,tilkjennes,estetisk
5,Schultz,kunsten,gjenbruk,kulturelle
6,Hamsuns,filosofi,begrep,universelle
7,Architecture,arkitektene,framstiller,moderne
8,kulturens,subjektet,rømmer,universelt
9,Profil,diskurs,,samlende


In [72]:
pm_total['combine'] = (pm_total.pmi/pm_total.score)

In [75]:
pm_total.sort_values(by='combine', ascending=False)

,freq,dist,weighted_distance,score,pmi,combine
postmoderne,94.0,6.18,580.950000,6.17,0.022975,0.003724
arkitekturen,38.0,4.76,180.750000,4.78,0.005876,0.001229
Samtiden,24.0,3.46,83.000000,3.54,0.003159,0.000892
Foucault,19.0,5.20,98.833333,5.22,0.001307,0.000250
epoker,8.0,4.25,34.000000,4.41,0.001008,0.000229
Torben,10.0,5.20,52.000000,5.23,0.001157,0.000221
kunsten,43.0,4.57,196.666667,4.59,0.000993,0.000216
oppløse,11.0,5.36,59.000000,5.37,0.001068,0.000199
arkitektur,31.0,6.56,203.250000,6.53,0.001280,0.000196
diskurser,10.0,7.20,72.000000,7.03,0.001223,0.000174


In [76]:
klassifikasjon(pm_total, by='combine', num=120, up=False).head(30)

,ukjent,subst,verb,adj
0,postmoderne,arkitekturen,oppløse,binære
1,Samtiden,epoker,avviser,universell
2,Foucault,kunsten,representerer,estetisk
3,Torben,arkitektur,vektlegger,psykoanalytisk
4,Habermas,diskurser,tilkjennes,arkitektoniske
5,Schultz,arkitektene,fortolket,kulturelle
6,Hamsuns,sannheter,framstiller,kulturell
7,viderefører,filosofi,gjenbruk,ideologisk
8,kulturens,søken,hevder,kritisk
9,Profil,diskurs,begrep,universelle
